# Prerequistes  

## Azure Key Vault 
1. Azure Key Vault stores Twitter developer account scretes: TWEET-CONSUMER-KEY, TWEET-CONSUMER-SECRET, TWEET-ACCESS-TOKEN, TWEET-ACCESS-SECRET
1. Add Azure Key Vault Linked Service "mylsKeyVault"
1. Add AKV access policy, secrete 'Get' permission

In [ ]:
# Primary storage info
account_name = 'twitterhashtagsandbox' # fill in your primary account name
hashtag = '#AzureDataFactory'
container_name = 'workspace' # fill in your primary file system name

In [3]:
from notebookutils import mssparkutils
from datetime import datetime, timedelta
import tweepy as tw

relative_path = 'myfolder' # fill in your relative folder path
adls_path = 'abfss://%s@%s.dfs.core.windows.net/%s' % (container_name, account_name, relative_path)

akv_linked_serivce = "mylsKeyVault"

auth = tw.OAuthHandler(mssparkutils.credentials.getSecretWithLS(akv_linked_serivce, "TWEET-CONSUMER-KEY"),mssparkutils.credentials.getSecretWithLS(akv_linked_serivce, "TWEET-CONSUMER-SECRET") )
auth.set_access_token(mssparkutils.credentials.getSecretWithLS(akv_linked_serivce, "TWEET-ACCESS-TOKEN"), mssparkutils.credentials.getSecretWithLS(akv_linked_serivce, "TWEET-ACCESS-SECRET") )
api = tw.API(auth, wait_on_rate_limit=True)

all_tweets = []
 
#main function that does all the work
def twitterQuery(search_words, all_tweets): 
    tweets = tw.Cursor(api.search, q=search_words).items()
        
    #potential complexity issue. the more tweet.attributes you pull the exponentially longer this takes
    hashtag_query = [[tweet.text, tweet.user.screen_name,tweet.user.location, tweet.created_at, tweet.retweet_count, tweet.favorite_count, tweet.entities['urls']] for tweet in tweets]
    #print(hashtag_query)

    for i in hashtag_query:
        url_status = 0
        i[3] = i[3].strftime('%Y-%m-%d')
        if i[6] != []:
            for j in range(len(i[6])):
                if i[6][j]['display_url']=='twitter.com/i/web/status/1…':
                    i[6] = i[6][j]['expanded_url']
                    url_status = 1
        if url_status == 0:
            i[6] = 'Please find it manually'

    all_tweets.extend(hashtag_query)
 
#hashtags to pass into main method
#hashtags = ["#DataFactory", "#AzureDataFactory"]
hashtags = [hashtag]

for i in hashtags:
    twitterQuery(i+" -filter:retweets", all_tweets)

from pyspark.sql import SparkSession
from pyspark.sql.types import *

today = datetime.today()
csv_path = adls_path + '/' + today.strftime("%Y%m%d")
print(csv_path)

df = spark.createDataFrame(all_tweets, ['tweet','user', 'location', 'created_at', 'retweet_count', 'favorites_count', 'link'])
df.orderBy([("created_at")], ascending=[0]) \
    .coalesce(1) \
    .dropDuplicates(subset = ['tweet','user']) \
    .coalesce(1).write.csv(csv_path, mode = 'overwrite', header = 'true')
